# Machine learning homework5
## 0. Preparation
### 0.1 Import required librarys

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from libsvm.python import svm
from libsvm.python import svmutil

### 0.2 Read training and testing data

In [ ]:
x_train = np.loadtxt('data/X_train.csv', delimiter=',')
x_test = np.loadtxt('data/X_test.csv', delimiter=',')
y_train = np.loadtxt('data/Y_train.csv', delimiter=',')
y_test = np.loadtxt('data/Y_test.csv', delimiter=',')

### 0.3 Construct problem with training data

In [ ]:
problem = svmutil.svm_problem(y_train, x_train)

## 1. Compare different kernels
### 1.1 Train with linear kernel
Use default parameters and quiet mode

In [ ]:
linear_param = svmutil.svm_parameter('-t 0 -q')
linear_model = svmutil.svm_train(problem, linear_param)

### 1.2 Train with polynomial kernel
Use default parameters and quiet mode

In [ ]:
poly_param = svmutil.svm_parameter('-t 1 -q')
poly_model = svmutil.svm_train(problem, poly_param)

### 1.3 Train with RBF kernel
Use default parameters and quiet mode

In [ ]:
rbf_param = svmutil.svm_parameter('-t 2 -q')
rbf_model = svmutil.svm_train(problem, rbf_param)

### 1.4 Predict testing data with three trained model

In [ ]:
linear_pred = svmutil.svm_predict(y_test, x_test, linear_model)
poly_pred = svmutil.svm_predict(y_test, x_test, poly_model)
rbf_pred = svmutil.svm_predict(y_test, x_test, rbf_model)

### 1.5 Visualize each kernel's accuracy

In [ ]:
names = ['Linear', 'Polynomial', 'RBF']
acc = [linear_pred[1][0], poly_pred[1][0], rbf_pred[1][0]]
plt.figure()
plt.bar(names, acc, width=0.5)
plt.title('Accuracy of different kernels with default parameters')
plt.xlabel('Kernels')
plt.ylabel('Accuracy')
plt.show()

## 2. Use C-SVC and tune with grid search
### 2.1 Search best parameter for linear kernel
Since linear kernel with C-SVC only have one parameter `C`, we search for that best C from 1e-5, 1e-4,..., 1e4 here

In [ ]:
log_c_range = np.arange(-5, 5, dtype=float)
c_range = 10 ** log_c_range
acc = []
for c in c_range:
    param = svmutil.svm_parameter(f'-t 0 -c {c} -v 5 -q')
    acc += [svmutil.svm_train(problem, param)]
print(f'Best C is 10^{log_c_range[np.argmax(acc)]}')
plt.figure()
plt.plot(log_c_range, acc, 'r-')
plt.title('Accuracy of different C')
plt.xlabel('C (power of 10)')
plt.ylabel('Accuracy')
plt.show()

## 2.2 Search best parameter for polynomial kernel
Polynomial kernel has its own parameter `gamma`, `degree` and `coef0`, so in this situatuion, we will search for combination of four parameters:
- C, power of 10, from 1e-5 to 1e0
- gamma, power of 10, from 1e-3 to 1e1
- degree, from 1 to 5
- coef0, from 0 to 4

First, we test for varying one parameter with other set to default value. Then we test for all combinations.
### 2.2.1 Vary C, other use default

In [ ]:
log_c_range = np.arange(-5, 0, dtype=float)
c_range = 10 ** log_c_range
acc = []
for c in c_range:
    param = svmutil.svm_parameter(f'-t 1 -c {c} -v 5 -q')
    acc += [svmutil.svm_train(problem, param)]
print(f'Best C is 10^{log_c_range[np.argmax(acc)]}')
plt.figure()
plt.plot(log_c_range, acc, 'r-')
plt.title('Accuracy of different C')
plt.xlabel('C (power of 10)')
plt.ylabel('Accuracy')
plt.show()

### 2.2.2 Vary gamma, other use default

In [ ]:
log_gamma_range = np.arange(-3, 2, dtype=float)
gamma_range = 10 ** log_gamma_range
acc = []
for gamma in gamma_range:
    param = svmutil.svm_parameter(f'-t 1 -g {gamma} -v 5 -q')
    acc += [svmutil.svm_train(problem, param)]
print(f'Best gamma is 10^{log_gamma_range[np.argmax(acc)]}')
plt.figure()
plt.plot(log_gamma_range, acc, 'r-')
plt.title('Accuracy of different gamma')
plt.xlabel('gamma (power of 10)')
plt.ylabel('Accuracy')
plt.show()

### 2.2.3 Vary degree, other use default

In [ ]:
degree_range = np.arange(1, 6, dtype=int)
acc = []
for degree in degree_range:
    param = svmutil.svm_parameter(f'-t 1 -d {degree} -v 5 -q')
    acc += [svmutil.svm_train(problem, param)]
print(f'Best coef0 is {degree_range[np.argmax(acc)]}')
plt.figure()
plt.plot(degree_range, acc, 'r-')
plt.title('Accuracy of different degree')
plt.xlabel('degree')
plt.ylabel('Accuracy')
plt.show()

### 2.2.4 Vary coef0, other use default

In [ ]:
coef0_range = np.arange(0, 5, dtype=int)
acc = []
for coef0 in coef0_range:
    param = svmutil.svm_parameter(f'-t 1 -r {coef0} -v 5 -q')
    acc += [svmutil.svm_train(problem, param)]
print(f'Best coef0 is {coef0_range[np.argmax(acc)]}')
plt.figure()
plt.plot(coef0_range, acc, 'r-')
plt.title('Accuracy of different coef0')
plt.xlabel('coef0')
plt.ylabel('Accuracy')
plt.show()

### 2.2.5 All combinations

In [ ]:
best_acc = 0
best_param = {}
for c in c_range:
    for gamma in gamma_range:
        for degree in degree_range:
            for coef0 in coef0_range:
                param = svmutil.svm_parameter(f'-t 1 -c {c} -g {gamma} -d {degree} -r {coef0} -v 5 -q')
                acc = svmutil.svm_train(problem, param)
                if acc > best_acc:
                    best_acc = acc
                    best_param['c'] = c
                    best_param['gamma'] = gamma
                    best_param['degree'] = degree
                    best_param['coef0'] = coef0
print(f'Parameter {best_param} achieve the best accuracy {best_acc}%')

## 2.3 Search best parameter for RBF kernel
RBF kernel has its own parameter `gamma`, so in this situatuion, we will search for combination of two parameters:
- C, power of 10, from 1e-5 to 1e0
- gamma, power of 10, from 1e-3 to 1e1

First, we test for varying one parameter with other set to default value. Then we test for all combinations.
### 2.3.1 Vary C, other use default

In [ ]:
log_c_range = np.arange(-5, 0, dtype=float)
c_range = 10 ** log_c_range
acc = []
for c in c_range:
    param = svmutil.svm_parameter(f'-t 2 -c {c} -v 5 -q')
    acc += [svmutil.svm_train(problem, param)]
print(f'Best C is 10^{log_c_range[np.argmax(acc)]}')
plt.figure()
plt.plot(log_c_range, acc, 'r-')
plt.title('Accuracy of different C')
plt.xlabel('C (power of 10)')
plt.ylabel('Accuracy')
plt.show()

### 2.3.2 Vary gamma, other use default

In [ ]:
log_gamma_range = np.arange(-3, 2, dtype=float)
gamma_range = 10 ** log_gamma_range
acc = []
for gamma in gamma_range:
    param = svmutil.svm_parameter(f'-t 2 -g {gamma} -v 5 -q')
    acc += [svmutil.svm_train(problem, param)]
print(f'Best gamma is 10^{log_gamma_range[np.argmax(acc)]}')
plt.figure()
plt.plot(log_gamma_range, acc, 'r-')
plt.title('Accuracy of different gamma')
plt.xlabel('gamma (power of 10)')
plt.ylabel('Accuracy')
plt.show()

### 2.3.3 All combinations

In [ ]:
best_acc = 0
best_param = {}
for c in c_range:
    for gamma in gamma_range:
        param = svmutil.svm_parameter(f'-t 1 -c {c} -g {gamma} -v 5 -q')
        acc = svmutil.svm_train(problem, param)
        if acc > best_acc:
            best_acc = acc
            best_param['c'] = c
            best_param['gamma'] = gamma
print(f'Parameter {best_param} achieve the best accuracy {best_acc}%')